In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pandas as pd
from sympy import *
import PIL
from PIL import ImageDraw
np.set_printoptions(threshold=5000000000)
import png
from scipy.ndimage.morphology import binary_fill_holes, binary_opening, binary_dilation
import cv2

In [2]:
def int64_feature(values):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))

def bytes_feature(values):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

def float_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def zoom_at(img, x, y, zoom):
    w, h = img.size
    zoom2 = zoom * 2
    img = img.crop((x - w / zoom2, y - h / zoom2, x + w / zoom2, y + h / zoom2))
    return img.resize((w, h), PIL.Image.LANCZOS)

def find_center(x_values, xmin, y_values, ymin):
    x1_index = np.where(x_values == xmin)[0][0]

    p1x = Point(x_values[x1_index], y_values[x1_index])
    x2_index = np.where(x_values == xmax)[0][0]
    p2x = Point(x_values[x2_index], y_values[x2_index])

    
    
    y1_index = np.where(y_values == ymin)[0][0]
    p1y = Point(x_values[y1_index], y_values[y1_index])
    y2_index = np.where(y_values == ymax)[0][0]
    p2y = Point(x_values[y2_index], y_values[y2_index])

            
    
    solutions = np.zeros(2)

        
    line1 = Line(Point(x_values[0],y_values[0]), Point(x_values[1],y_values[1]))
    line2 = Line(Point(x_values[2],y_values[2]), Point(x_values[3],y_values[3]))
    line1_coeff = np.asarray(list(line1.coefficients)).astype(np.float)
    line2_coeff = np.asarray(list(line2.coefficients)).astype(np.float)
    solutions = np.array([-1*line1_coeff[2], -1*line2_coeff[2]])
    line1_coeff = np.delete(line1_coeff, 2)
    line2_coeff = np.delete(line2_coeff, 2)
    b = np.array([line1_coeff, line2_coeff])
    row_sol, col_sol = np.linalg.lstsq(b, solutions, rcond = None)[0]
    center = Point(row_sol, col_sol)
        
    return center

In [3]:
df = pd.read_csv("C:\\Temp\\nitish\\Tensorflow\\workspace\\training_demo\\annotations\\train2.csv", usecols=["File_name", "Patient_index", "Study_index", "Series_ID", "Key_slice_index", "Measurement_coordinates", "Bounding_boxes", "Lesion_diameters_Pixel_", "Normalized_lesion_location", "Coarse_lesion_type", "Possibly_noisy", "Slice_range", "Spacing_mm_px_", "Image_size", "DICOM_windows", "Patient_gender", "Patient_age", "Train_Val_Test"])
df.set_index("File_name", inplace = True)

tfrecord_train_filename = "C:\\Temp\\nitish\\Tensorflow\\workspace\\training_demo\\annotations\\train.record"
tfrecord_test_filename = "C:\\Temp\\nitish\\Tensorflow\\workspace\\training_demo\\annotations\\test.record"
test_image_path = "C:\\Temp\\nitish\\Tensorflow\\workspace\\training_demo\\images\\test\\images\\"
train_image_path = "C:\\Temp\\nitish\\Tensorflow\\workspace\\training_demo\\images\\train\\images\\"
test_mask_path = "C:\\Temp\\nitish\\Tensorflow\\workspace\\training_demo\\images\\test\\binary_masks\\"
train_mask_path = "C:\\Temp\\nitish\\Tensorflow\\workspace\\training_demo\\images\\train\\binary_masks\\"
ex_path = "C:\\Temp\\nitish\\Tensorflow\\workspace\\training_demo\\images\\Extracted_Images\\"

os.chdir(train_image_path)
print(os.getcwd())

C:\Temp\nitish\Tensorflow\workspace\training_demo\images\train\images


In [62]:
#writer = tf.python_io.TFRecordWriter(tfrecord_train_filename)
count = 0
dupList = df.index.get_duplicates()

#iterate through pictures
for image in os.listdir():

    x,y = image.split(".")
    image_name = str(train_image_path + x + "." + y)
    img = np.array(PIL.Image.open(image_name))
    test = np.zeros((512,512))
    gith = np.stack((img,test), axis=2)
    #print(gith.shape)
    #print(gith[200:300, 200:300, :])
    #break
    """
    img -= 32768
    
    #PIL.Image.fromarray(img).show()
    win = [-1024, 3071]
    im1 = img.astype(float)
    im1 -= win[0]
    im1 /= win[1] - win[0]
    im1[im1 > 1] = 1
    im1[im1 < 0] = 0
    im1 *= 255
    img = im1
    PIL.Image.fromarray(img).show()
    print(img[100:400, 100:400])

    break
    """
    #load images and masks
    
    if img.shape[0] == 512 and not(image in dupList):
        
        
        mask_name = str(train_mask_path + x + "_mask" + "." + y)
        mask = np.array(PIL.Image.open(mask_name))
        height = img.shape[0]
        width = img.shape[1]
        
        
        string_pixels = str(df.loc[image]["Bounding_boxes"])
        raw_pixels = np.asarray(string_pixels.split(','))
        x_values = raw_pixels[0:3:2].astype(np.float)
        y_values = raw_pixels[1:4:2].astype(np.float)
        
        xmax = np.amax(x_values)
        xmin = np.amin(x_values)
        ymax = np.amax(y_values)
        ymin = np.amin(y_values)
        
        
        #check image viability
        if xmin > 512 or xmax > 512 or ymin > 512 or ymax > 512:
            continue
        
        #converts to 16-bit as a test
        
        """
        #16-BIT CONVERTER
        img = img.astype(np.float32, copy=False)-32768
        string_dicom = str(df.loc[image]["DICOM_windows"])
        raw_dicom = np.asarray(string_dicom.split(','))
        dmin = raw_dicom[0].astype(np.float)
        dmax = raw_dicom[1].astype(np.float)
        row , col = img.shape
        for r in range(0, row):
            for c in range(0, col):
                val = int(img[r,c])
                img[r,c] = int(min(65535, max(0, (val-dmin)/(dmax-dmin)*65535)))
        img = img.astype('uint16')
        """
        #pimg = PIL.Image.fromarray(img).convert('RGB')
         
        #pimg.save(temp_path, format='png')
        
        #new_img = zoom_at(pimg, center.x, center.y, 3)
        #new_img.show()
        
        """
        
        print(img)
        #img = np.dstack((img,img,img))
        
        #test_image = PIL.Image.fromarray(img)
        #test_image.show()
        
        print(img.shape)
        temp_path = "C:\\Temp\\nitish\\Tensorflow\\workspace\\training_demo\\images\\temp\\temp.png"
        temp_masks_path = "C:\\Temp\\nitish\\Tensorflow\\workspace\\training_demo\\images\\temp_masks\\temp_mask.png"
         
        writerx = png.Writer(width=img.shape[1], height=img.shape[0], bitdepth=16)
        with open(temp_path, 'wb') as f:
            z2list = img.reshape(-1, img.shape[1]*img[2]).tolist()
            #z2list = im1.tolist()
            writerx.write(f, z2list)
        #img.save(temp_path, format='png')
        
        """
        
        
        #image slicer for testing purposes
        """
        
        masker = img > 32000
        masker = binary_opening(masker, structure = np.ones((7,7)))
        
        masker = masker.astype(dtype=np.int32)
        
        idx = np.nonzero(masker)
    
        u = max(0, idx[0].min())
        d = min(masker.shape[0] - 1, idx[0].max())
        l = max(0, idx[1].min())
        r = min(masker.shape[1] - 1, idx[1].max())
        cut = [u, d, l, r]
        cut = [0, 511, 0, 511]
        """
        
        #HU conversion
        """
        img = img.astype(np.float32, copy=False)-32768
        win = [-1024, 3071]
        im1 = img.astype(float)
        im1 -= win[0]
        im1 /= win[1] - win[0]
        im1[im1 > 1] = 1
        im1[im1 < 0] = 0
        im1 *= 65535
        """

        #im1 = img[cut[0]:cut[1] + 1, cut[2]:cut[3] + 1]
        #mask = mask[cut[0]:cut[1] + 1, cut[2]:cut[3] + 1]
        
        """
        #convert to 8-bit
        im1 = im1.astype(np.float32, copy=False)-32768
        string_dicom = str(df.loc[image]["DICOM_windows"])
        raw_dicom = np.asarray(string_dicom.split(','))
        dmin = raw_dicom[0].astype(np.float)
        dmax = raw_dicom[1].astype(np.float)
        row , col = im1.shape
        for r in range(0, row):
            for c in range(0, col):
                val = int(im1[r,c])
                im1[r,c] = int(min(255, max(0, (val-dmin)/(dmax-dmin)*255)))
        im1 = im1.astype('uint8')
        """
        
        
        #print(c)
        #print(ymax)
        
        """
        xmin = xmin - cut[2]
        xmax = xmax - cut[2]
        ymin = ymin - cut[0]
        ymax = ymax - cut[0]
        """

        #RESIZER. This code block will normalize pixel spacing.
        
        
        spacing = str(df.loc[image]["Spacing_mm_px_"])
        slice_prop = np.asarray(spacing.split(','))[2].astype(np.float)
        
        """
        if spacing is not None:  # spacing adjust, will overwrite simple scaling
            im_scale = float(spacing) / 0.8
        
        max_shape = 512*im_scale
        
        if max_shape > 512:
            im_scale1 = float(512) / max_shape
            im_scale *= im_scale1
        
        if im_scale != 1:
            im1 = cv2.resize(im1, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR)
            mask = cv2.resize(mask, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR)
            xmax *= im_scale
            xmin *= im_scale
            ymax *= im_scale
            ymin *= im_scale
            #print(xmin, xmax, im1.shape)
        """
        
        im_scale = 1
        
        
        idx = int(df.loc[image]["Key_slice_index"])
        ranger = str(df.loc[image]["Slice_range"])
        minslice, maxslice = ranger.split(',')
        minslice = int(minslice)
        maxslice = int(maxslice)
        #print(minslice, maxslice)
        #break
        a,b,m,d = x.split("_")
        target_dir = a + "_" + b + "_" + m
        #To generate 3D images, image propagation was implemented to look spatially above and below each key slice. 
        
        ims = [img]
        number_props = 3
        # find neighboring slices of im_cur
        rel_pos = float(2) / slice_prop
        a = rel_pos - np.floor(rel_pos)
        b = np.ceil(rel_pos) - rel_pos
        
        if a == 0:  # required SLICE_INTV is a divisible to the actual slice_intv, don't need interpolation
                       
            for p in range(number_props):
            
                prev_index = idx - (rel_pos * (p+1))
                if prev_index < minslice:
                    prev_index = minslice
                prev_index = int(prev_index)
                im_prev =  np.array(PIL.Image.open(str(ex_path + target_dir + "\\" + str(f'{prev_index:03}' + ".png"))))
            
                next_index = idx + (rel_pos * (p+1))
                if next_index > maxslice:
                    next_index = maxslice
                next_index = int(next_index)
                im_next =  np.array(PIL.Image.open(str(ex_path + target_dir + "\\" + str(f'{next_index:03}' + ".png"))))
            
                ims = [im_prev] + ims + [im_next]    
                
                
                
        else:
            
            for p in range(number_props):
                intv1 = rel_pos*(p+1)
                
                prev_index_1 = idx - np.ceil(intv1)
                if prev_index_1 < minslice:
                    prev_index_1 = minslice
                prev_index_1 = int(prev_index_1)
                
                prev_index_2 = idx - np.floor(intv1)
                if prev_index_2 < minslice:
                    prev_index_2 = minslice
                prev_index_2 = int(prev_index_2)
                
                slice1 = np.array(PIL.Image.open(str(ex_path + target_dir + "\\" + str(f'{prev_index_1:03}' + ".png"))))
                slice2 = np.array(PIL.Image.open(str(ex_path + target_dir + "\\" + str(f'{prev_index_2:03}' + ".png"))))
                im_prev = a * slice1 + b * slice2  # linear interpolation
                
                

                next_index_1 = idx + np.ceil(intv1)
                if next_index_1 > maxslice:
                    next_index_1 = maxslice
                next_index_1 = int(next_index_1)
                
                next_index_2 = idx + np.floor(intv1)
                if next_index_2 > maxslice:
                    next_index_2 = maxslice
                next_index_2 = int(next_index_2)
                
                slice1 = np.array(PIL.Image.open(str(ex_path + target_dir + "\\" + str(f'{next_index_1:03}' + ".png"))))
                slice2 = np.array(PIL.Image.open(str(ex_path + target_dir + "\\" + str(f'{next_index_2:03}' + ".png"))))
                im_next = a * slice1 + b * slice2  # linear interpolation

                ims = [im_prev] + ims + [im_next]
        
        
        ims = [im.astype(float) for im in ims]
        im = cv2.merge(ims)
        im = im.astype(np.float32, copy=False)-32768
        
        win = [-1024, 3071]
        im = im.astype(float)
        im -= win[0]
        im /= win[1] - win[0]
        im[im > 1] = 1
        im[im < 0] = 0
        im *= 255
        #print(im[200:300,200:300,0])
        
        
        
        
        break
        
        
        #print(ims[0])
        
        
        eight = (ims/256.0).astype(int)
        #plt.plot(eight[0, :, :])
        break
        
        count+=1
        if count == 5:
            break
        continue
        #break
        
        
        #convert spatial slices to usable format
        
        #imga = imga[cut[0]:cut[1] + 1, cut[2]:cut[3] + 1]
        #imgb = imgb[cut[0]:cut[1] + 1, cut[2]:cut[3] + 1]
        
        #if im_scale != 1:
            #imga = cv2.resize(imga, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR)
            #imgb = cv2.resize(imgb, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR)
        
        
        
        """
        imga = imgabove.astype(np.float32, copy=False)-32768
        win = [-1024, 3071]
        imga = imga.astype(float)
        imga -= win[0]
        imga /= win[1] - win[0]
        imga[imga > 1] = 1
        imga[imga < 0] = 0
        imga *= 65535
        #print(imga[0:10, :])
        imga = imga[c[0]:c[1] + 1, c[2]:c[3] + 1]
        if im_scale != 1:
            imga = cv2.resize(imga, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR)
       
        imga = np.matrix.round(imga)
        imga = imga.astype(dtype=int)
        
        imgb = imgbelow.astype(np.float32, copy=False)-32768
        win = [-1024, 3071]
        imgb = imgb.astype(float)
        imgb -= win[0]
        imgb /= win[1] - win[0]
        imgb[imgb > 1] = 1
        imgb[imgb < 0] = 0
        imgb *= 65535
        imgb = imgb[c[0]:c[1] + 1, c[2]:c[3] + 1]
        if im_scale != 1:
            imgb = cv2.resize(imgb, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR)
        
        imgb = np.matrix.round(imgb)
        imgb = imgb.astype(dtype=int)
        
        """
        
        
        #save 3D image so it can be unpacked into Bytes later
        print(imgb.shape, imga.shape, im1.shape)
        final_image = cv2.merge([imgb, im1, imga])
        im = PIL.Image.fromarray(final_image)
        im.save(temp_path)

        
        #16 bit writing tester
        """
        writerx = png.Writer(width=final_image.shape[1], height=final_image.shape[0], bitdepth=16)
        #print(image)
        with open(temp_path, 'wb') as f:
            z2list = final_image.reshape(-1, final_image.shape[1]*final_image.shape[2]).tolist()
            #z2list = im1.tolist()
            writerx.write(f, z2list)
        """
        
        #generate normalized lesion bbox coordinates
        
        PIL.Image.fromarray(mask).save(temp_masks_path)
        #PIL.Image.fromarray(mask).show()
        
        xmins = float(xmin/im1.shape[1])
        ymins = float(ymin/im1.shape[0])
        xmaxs = float(xmax/im1.shape[1])
        ymaxs = float(ymax/im1.shape[0])
        

        #detect erroneous images
        if xmaxs > 1 or ymaxs > 1:
            print(count)
            continue
                
        #lesion classification  
        """
        lesionNum = df.loc[image]["Coarse_lesion_type"]
        if(lesionNum == 1):
            lesionType = 'bone'
        elif(lesionNum == 2):
            lesionType = 'abdomen'
        elif(lesionNum == 3):
            lesionType = 'mediastinum'
        elif(lesionNum == 4):
            lesionType = 'liver'
        elif(lesionNum == 5):
            lesionType = 'lung'
        elif(lesionNum == 6):
            lesionType = 'kidney'
        elif(lesionNum == 7):
            lesionType = 'soft_tissue'
        elif(lesionNum == 8):
            lesionType = 'pelvis'
        elif(lesionNum == -1):
            lesionNum = 9
            lesionType = 'unidentified'
        """
        lesionType = 'lesion'
        lesionNum = 1
        
        #img_raw = tf.compat.as_bytes(bytes(img))
        
        #make raw image
        
        with tf.gfile.GFile(temp_path, 'rb') as fid:
            img_raw = fid.read()  
        with tf.gfile.GFile(temp_masks_path, 'rb') as fid:
            mask_raw = fid.read()
        
        
        #create record dictionary
        feature_dict = {
        
            'image/height': int64_feature(im1.shape[0]),
            'image/width': int64_feature(im1.shape[1]),
            'image/format': bytes_feature('png'.encode('utf8')),
            'image/filename': bytes_feature(image.encode('utf8')),
            'image/source_id': bytes_feature(image.encode('utf8')),
            'image/encoded': bytes_feature(img_raw),
            'image/object/bbox/xmin': float_feature(xmins),
            'image/object/bbox/xmax': float_feature(xmaxs),
            'image/object/bbox/ymin': float_feature(ymins),
            'image/object/bbox/ymax': float_feature(ymaxs),
            'image/object/class/text': bytes_feature(lesionType.encode('utf8')),
            'image/object/class/label': int64_feature(1),
            'image/object/mask': bytes_feature(mask_raw),
        }
        
        
        count += 1
        
        #make record and write to file
        
        #example = tf.train.Example(features=tf.train.Features(feature = feature_dict))
        #writer.write(example.SerializeToString()) 
        
        if count == 0:
            break


print(count)
#close the writer
#writer.close()

c:\program files\python\python36\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 'get_duplicates' is deprecated and will be removed in a future release. You can use idx[idx.duplicated()].unique() instead
  This is separate from the ipykernel package so we can avoid doing imports until
